In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
from drive.ML_Study import * # 让driver连接云盘的ML_Study

In [0]:
function ClickConnect(){
  console.log("Working"); 
  document
    .querySelector("#top-toolbar > colab-connect-button")
    .shadowRoot
    .querySelector("#connect")
    .click()
}
 
setInterval(ClickConnect,60000)

In [0]:
pip install transformers

     |████████████████████████████████| 645kB 12.6MB/s 
     |████████████████████████████████| 890kB 48.5MB/s 
     |████████████████████████████████| 1.1MB 42.6MB/s 
     |████████████████████████████████| 3.8MB 38.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=fd7a9f6fc3b7d900a06c4b5afde4510b7487eec3ef2382dcf011a11fb73bd280
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
import json
import numpy as np
from tqdm import tqdm
import os
import tensorflow as tf
from transformers import *

In [0]:
maxlen=128

In [0]:
print('Lodaing BERT')
pretrainedModel_path = 'drive/ML_Study/bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(os.path.join(pretrainedModel_path, 'vocab.txt'))
config = BertConfig.from_json_file(os.path.join(pretrainedModel_path, 'config.json'))
print('==========END==========')

Lodaing BERT


Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


==========END==========


In [0]:
# 生成新的SPO
def load_data(path):
    D = []
    with open(path, encoding='utf-8') as f:
        for l in f.readlines():
            l = json.loads(l)
            D.append(
             l['text']
            )
    return D

# 得到s,p对应值的初始下标
def search(pattern, sequence):
    n = len(pattern)
    for i in range(len(sequence)):
        if sequence[i:i + n] == pattern:
            return i
    return -1

# 寻找特殊符号的位置
def search_special_label(pattern, sequence):
    rt = []
    for i, sid in enumerate(sequence):
        if sid == pattern:
            rt.append(i)
    return rt

In [0]:
# category mapping
def get_mapping(mapping_path):
    predicate2id, id2predicate = {}, {}
    with open(mapping_path, encoding='utf-8') as f:
        for l in f.readlines():
            l = json.loads(l)
            for key in l['object_type']:
                p = (l['predicate']+'_'+key)
                if p not in predicate2id:
                    id2predicate[len(predicate2id)] = p
                    predicate2id[p] = len(predicate2id)
    return predicate2id, id2predicate
    
mapping_path = 'drive/ML_Study/Baidu_ERE_Data/schema.json'

predicate2id, id2predicate = get_mapping(mapping_path)

In [0]:
# get BIO-relation label mapping
# get predicate mapping
special_labels = ['[PAD]','[CLS]','[SEP]','[UNK]']
special_labels_token = [0,101,102,100]
# All label
labels = []
for l in predicate2id.keys():
    l = 'B-Sub-'+l
    labels.append(l)
for l in predicate2id.keys():
    l = 'B-Obj-'+l
    labels.append(l)
labels.append('I')
labels.append('O')
for l in special_labels:
    labels.append(l)
    
# Bilabel Mapping
id2BIO = {}
BIO2id = {}
for i,label in enumerate(labels):
    id2BIO[i] = label
    BIO2id[label] = i

In [0]:
def get_input(data,tokenizer,maxlen=128):
    token_ids, segment_ids, position_ids= [], [], []
    for t in data:
        token_id, segment_id, position_id = tokenizer.encode_plus(t, max_length=maxlen,pad_to_max_length=True).values()
        token_ids.append(token_id)
        segment_ids.append(segment_id)
        position_ids.append(position_id)
    return [token_ids,position_ids,segment_ids]
#     return {'input_ids':tf.convert_to_tensor(tf.constant(token_ids)),
#             'attention_mask':tf.convert_to_tensor(tf.constant(position_ids)),
#             'token_type_ids':tf.convert_to_tensor(tf.constant(segment_ids))}

In [0]:
# test_data = load_data('drive/ML_Study/Baidu_ERE_Data/test1_data.json')
test_data = load_data('drive/ML_Study/Baidu_ERE_Data/test2_data.json')

In [0]:
print('Contructing Data')
# train_xy = get_input(train_data,tokenizer,predicate2id,maxlen=128)
test_input = get_input(test_data,tokenizer,maxlen=maxlen)
test_dataset = {'input_ids':tf.convert_to_tensor(tf.constant(test_input[0])),
            'attention_mask':tf.convert_to_tensor(tf.constant(test_input[1])),
            'token_type_ids':tf.convert_to_tensor(tf.constant(test_input[2]))}
print('==========END==========')

Contructing Data
==========END==========


In [0]:
class SPO_Model(TFBertPreTrainedModel):
    def __init__(self, config, *inputs, **kwargs):
        super().__init__(config, *inputs, **kwargs)
        self.bert = TFBertMainLayer(config, name="bert")
        self.dropout = tf.keras.layers.Dropout(config.hidden_dropout_prob)
        self.BiLSTM = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256,return_sequences=True))
        self.classifier = tf.keras.layers.Dense(116,activation=tf.nn.sigmoid,kernel_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.05))
    def call(self, inputs, **kwargs):
        bert_outputs = self.bert(inputs, **kwargs)
        sequence_output = bert_outputs[0]
        x = self.dropout(sequence_output, training=kwargs.get("training", False))
        x = self.BiLSTM(x)
        x = self.classifier(x)
        x = tf.keras.layers.Lambda(lambda x: x**(1/0.95))(x) #对BinaryEntropy进行放大处理，以排除人工标注错误，假设95%的数据都是正确的

        # CRF Layer 其实不适合该模型
        return x

In [0]:
spo_model = SPO_Model.from_pretrained(pretrainedModel_path,config=config)

In [0]:
spo_model.load_weights('drive/ML_Study/saved_Model/weight_200410v4/spo_model_20epochs.ckpt')

In [0]:
# m = SPO_Model.from_pretrained(pretrainedModel_path,config=config)

In [0]:
# m.load_weights('drive/ML_Study/saved_Model/weight_200410v3/spo_model_15epochs.ckpt')

In [0]:
# label -> SPOes
def predict_SPOes(token_id, label): # zip(token_id, label_pred)
  SPOes=[]
  sub_text_p, sub_category_p = np.where(label[:,:55]>0.5) 
  obj_text_p, obj_category_p = np.where(label[:,55:110]>0.5) # obj_category_p 自动减去48!! 返回的值是从 0 -> 47!
  i_text_p = sorted(np.where(label[:,110]>0.5)) # I 的对应下标
  sub_info = list(zip(sub_text_p, sub_category_p))
  obj_info = list(zip(obj_text_p, obj_category_p))

  SPOes=[]

  for sub in sub_info:
    sub_start = sub[0]
    sub_end = sub_start
    for I_i in i_text_p[0]: # 确定 sub 的长度
        if I_i == (sub_end + 1):
            sub_end += 1
    for obj in obj_info: 
        if sub[1] == (obj[1]):
            obj_start = obj[0]
            obj_end =  obj_start
            for I_i in i_text_p[0]: # 确定 obj 的长度
                if I_i == (obj_end + 1):
                    obj_end += 1
            spo = (
                (tokenizer.decode(token_id[sub_start: sub_end+1])).replace(" ",""),
                id2predicate[sub[1]],
                (tokenizer.decode(token_id[obj_start: obj_end+1])).replace(" ","")
                )
            SPOes.append(spo)
  return SPOes

In [0]:
def get_relation_pattern(mapping_path):
    relation_pattern = {}
    with open(mapping_path,encoding='utf-8') as f:
        for l in f.readlines():
            l = json.loads(l)
            relation_pattern[l['predicate']] = (l['object_type'],l['subject_type'])
    return relation_pattern

relation_pattern = get_relation_pattern(mapping_path)

In [0]:
# SPOes -> final output
def get_spo_list(SPOes,relation_pattern):
    complex_relations = ['配音','上映时间','票房','获奖','饰演']
    spo_list= []
    for i1, spo in enumerate(SPOes):
        p = spo[1].split('_') # 切开predicate
        if p[1]  == '@value':
            obj={}
            obj[p[1]] = spo[2]
            obj_type = {}
            obj_type[p[1]] = relation_pattern[p[0]][0][p[1]] # object_type
            if p[0] in complex_relations: # 如果是复杂关系就找其子预测       
                for i2, spo_c in enumerate(SPOes):
                    if i1 == i2:
                        continue
                    p_c = spo_c[1].split('_')
                    if p_c[0] == p[0]:
                        obj[p_c[1]] = spo_c[2]
                        obj_type[p_c[1]] = relation_pattern[p[0]][0][p_c[1]] # object_type
        
            spo_list.append({
                        'predicate':p[0],
                        'object_type':obj_type,# 只保留存在的key
                        'subject_type':relation_pattern[p[0]][1],
                        'object':obj,
                        'subject':spo[0]
            })
    return spo_list

In [0]:
rt_labels  = []

for i in test_input[0]:
  rt_labels.append(spo_model(tf.constant([i]))[0])
  

In [0]:
f = open('drive/test2_pred.json', 'w', encoding='utf-8')

for text, token_id, label in zip(test_data,test_input[0], rt_labels):
    SPOes = predict_SPOes(token_id, label.numpy())
    out = json.dumps({
        'text': text,
        'spo_list':get_spo_list(SPOes,relation_pattern)
    },ensure_ascii=False)
    f.write(out + '\n')
f.close()

In [0]:
def get_input(data,tokenizer,predicate2id,maxlen=128):
    special_labels_token = [0,101,102,100]
    token_ids, segment_ids, position_ids= [], [], []
    spo_labels = []
    for d in data:
        # get X 
        token_id, segment_id, position_id = tokenizer.encode_plus(d['text'], max_length=maxlen,pad_to_max_length=True).values()
        token_ids.append(token_id)
        segment_ids.append(segment_id)
        position_ids.append(position_id)
        # get Y
        # 116 = 2R + 4 + 2, R=55
        # I:110, O:111
        label = np.zeros((116,maxlen)) # len(labels), maxlen
        label[111,:] = 1 # 先设置所有标签都是 O
        for s, p, o in d['spo_list']:
            s = tokenizer.encode(s)[1:-1]# 101 是 CLS 102 是ESP 这里只是为了得到s内部值的ID
            p_i = predicate2id[p] # 对category进行映射得到对应的ID
            o = tokenizer.encode(o)[1:-1] 
            s_i = search(s, token_id) # 对text进行映射
            o_i = search(o, token_id)
            # Sub
            label[p_i,s_i] = 1
            label[111,s_i] = 0
            for i in range(1,len(s)):
                s_I_i = s_i + i
                label[110,s_I_i] = 1
                label[111,s_I_i] = 0
            # Obj
            label[55+p_i,o_i] = 1
            label[111,o_i] = 0
            for i in range(1,len(o)):
                o_I_i = o_i + i
                label[110,o_I_i] = 1
                label[111,o_I_i] = 0
        # specail label
        for i, sp in enumerate(special_labels_token):
            sp_is = search_special_label(sp, token_id)
            for sp_i in sp_is:
                label[112+i,sp_i] = 1
                label[111,sp_i] = 0
                
        spo_labels.append(label.T)
    # return [
    #         tf.convert_to_tensor(tf.constant(token_ids)),
    #         tf.convert_to_tensor(tf.constant(position_ids)),
    #         tf.convert_to_tensor(tf.constant(segment_ids)),
    #         tf.convert_to_tensor(tf.constant(spo_labels))
    # ]
    return [token_ids, position_ids, segment_ids, spo_labels] 

In [0]:
def load_datav2(path):
    D = []
    with open(path, encoding='utf-8') as f:
        for l in f.readlines():
            l = json.loads(l)
            D.append({
                'text': l['text'],
                'spo_list': [
                    (spo['subject'], (spo['predicate']+'_'+key), spo['object'][key])
                    for spo in l['spo_list'] for key in spo['object'] 
                ]
            })
    return D

In [0]:
sample_train_data = load_datav2('drive/ML_Study/Baidu_ERE_Data/sample_data.json')

In [0]:
sample_train_xy = get_input(sample_train_data,tokenizer,predicate2id,maxlen=128)

In [0]:
a = sample_train_data[18]['text']
b = sample_train_data[18]['spo_list']
print(a)
print(b)

1990年2月，苏有朋和陈志朋、吴奇隆联袂出演了电影《好小子之游侠儿》，在影片中扮演了一个富有科技头脑，又带有稚气的高中生“小乖”，并合唱影片主题曲《游侠儿》，本片获得1990年度台湾“十大最高票房电影”之一
[('苏有朋', '获奖_onDate', '1990年'), ('苏有朋', '获奖_@value', '十大最高票房电影')]


In [0]:
rt = predict_SPOes(a_token, label[0])

In [0]:
a_token = tokenizer.encode(a, max_length=128, pad_to_max_length=True)
print(a_token)

[101, 8431, 2399, 123, 3299, 8024, 5722, 3300, 3301, 1469, 7357, 2562, 3301, 510, 1426, 1936, 7384, 5468, 6146, 1139, 4028, 749, 4510, 2512, 517, 1962, 2207, 2094, 722, 3952, 899, 1036, 518, 8024, 1762, 2512, 4275, 704, 2815, 4028, 749, 671, 702, 2168, 3300, 4906, 2825, 1928, 5554, 8024, 1348, 2372, 3300, 4928, 3698, 4638, 7770, 704, 4495, 100, 2207, 731, 100, 8024, 2400, 1394, 1548, 2512, 4275, 712, 7579, 3289, 517, 3952, 899, 1036, 518, 8024, 3315, 4275, 5815, 2533, 8431, 2399, 2428, 1378, 3968, 100, 1282, 1920, 3297, 7770, 4873, 2791, 4510, 2512, 100, 722, 671, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [0]:
label = spo_model(tf.constant([a_token]))

In [0]:
get_spo_list(rt,relation_pattern)

[{'object': {'@value': '十 大 最 高 票 房 电 影 [UNK]', 'onDate': '1990'},
  'object_type': {'@value': '奖项', 'onDate': 'Date'},
  'predicate': '获奖',
  'subject': '苏 有 朋',
  'subject_type': '娱乐人物'},
 {'object': {'@value': '台 湾 [UNK] 十 大 最 高 票 房 电 影 [UNK]', 'onDate': '1990'},
  'object_type': {'@value': '奖项', 'onDate': 'Date'},
  'predicate': '获奖',
  'subject': '苏 有 朋',
  'subject_type': '娱乐人物'}]